### BIM to GIS Conversion

#### This Notebook imports revit files from BIM models and convert them to GIS geodatabase in prepartion for monitoring
#### Please implement the following steps:
1. Define Projection to PRS92
2. Export Feature to file geodatabase
3. Merge all feature layers
4. Delete unwanted fields
5. Add Fields
6. Calculate Field for Type
7. Calculate Field for SubType
8. Delete Fields

In [36]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'
#workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'


In [37]:
# First, makre sure to Add to Current Map all the models:
## e.g. 'StructuralColumns', 'Roofs'

# Name of Depot Building
dName = "WS"
fDataset = "Depot_Building"
    
path = os.path.join(workspace, fDataset)

In [38]:
# 1. Define Projection to PRS92
## Add revit files to the Contents panel
## Note that 'Floors' in the BIM Model includes both slab and floors together.

columnA = r'Columns'
floor = r'Floors'
roof = r'Roofs'
framing = r'StructuralFraming'
foundation = r'StructuralFoundation'
column = r'StructuralColumns'

# Note that if we import from '-MOD-A-000001.rvt', it uses different 'BldgLevel' domain
# names and levels fro '-MOD-ST-000001.rvt'.
# It may be better for the GIS model to only utilize '-MOD-ST-000001.rvt' (i.e., drop 'roof')

if dName in ("LRS", "WS"):
    in_features = [floor, roof, framing, foundation, column]
elif dName in ("SanFernando"):
    in_features = [columnA, framing, foundation, column]
elif dName in ("Clark", "Angeles"):
    in_features = [floor, framing, foundation, column]
elif dName in ("CMV", "LOS"):
    in_features = [framing, foundation, column]


sr = arcpy.SpatialReference("PRS 1992 Philippines Zone III")

for fl in in_features:
    # Define Projection first
    arcpy.DefineProjection_management(fl, sr)    


In [39]:
# 2. Export to file geodatabase

arcpy.FeatureClassToGeodatabase_conversion(in_features, path)


<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\Depot_Building'>

In [40]:
# 3. Merge all feature layers
arcpy.env.workspace = path
output_feature = "N2_DEP_" + dName + "_merged"
arcpy.Merge_management(in_features, output_feature,"","")

<Result 'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb\\Depot_Building\\N2_DEP_WS_merged'>

In [41]:
# 4. Delete Fields
## 4.1 Obtain fields to be kept
fieldList = [f.name for f in arcpy.ListFields(output_feature)]

# find OBJECTID, as this is not deletable
reg = re.compile(r'OBJECTID*|Shape|Shape_Length|Shape_Area|Shape.STArea()|Shape.STLength()|GlobalID')
objField = list(filter(reg.match, fieldList))
keepFields = objField + ['Category', 'Family', 'FamilyType','ObjectID', 'ObjectId', 'Bldg_Name', 'BldgLevel', 'OmniClassDescription', 'BldgLevel_Desc', 'DocName', 'Discipline']

# Drop fields
dropFields = [e for e in fieldList if e not in keepFields]

## 4.2 Delete fields from feature layer
arcpy.DeleteField_management(output_feature, dropFields)

<Result 'N2_DEP_WS_merged'>

In [42]:
# 5. Add Fields
addFields = ['Name', 'Type', 'SubType', 'Status', 'StartDate', 'TargetDate', \
             'StructureType', 'StructureLevel', 'P6ID', 'Comment', \
            'GridX', 'GridY', 'CP', 'ID'] # ID is for bored piles

for field in addFields:
    if field in ('StartDate', 'TargetDate'):
        arcpy.AddField_management(output_feature, field, "DATE", "", "", "", field, "NULLABLE", "")
    elif field in ('P6ID', 'Comment', 'GridX', 'GridY', 'CP', 'Name', 'ID'):
        arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
    else:
        arcpy.AddField_management(output_feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [43]:
# Set the initial status = 'To be Constructed'
with arcpy.da.UpdateCursor(output_feature, ['Status']) as cursor:
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)

In [44]:
# Enter Depot building name
with arcpy.da.UpdateCursor(output_feature, ['Name']) as cursor:
    for row in cursor:
        if dName == "DEP":
            row[0] = "DEP"
        elif dName == "OCC":
            row[0] = "OCC"
        elif dName == "WS":
            row[0] = "WS"
        elif dName == "LRS":
            row[0] = "LRS"
        elif dName == "UCS":
            row[0] = "UCS"
        elif dName == "URS":
            row[0] = "URS"
        elif dName == "WRS":
            row[0] = "WRS"
        elif dName == "CMV":
            row[0] = "CMV"
        elif dName == "SCS":
            row[0] = "SCS"
        elif dName == "FPH1":
            row[0] = "FPH1"
        elif dName == "WOS":
            row[0] = "WOS"
        elif dName == "LOS":
            row[0] = "LOS"
        elif dName == "DHS":
            row[0] = "DHS"
        elif dName == "LGS":
            row[0] = "LGS"
        elif dName == "WGS":
            row[0] = "WGS"
        elif dName == "TGB":
            row[0] = "TGB"
        elif dName == "TMO":
            row[0] = "TMO"
        elif dName == "MCS":
            row[0] = "MCS"
        elif dName == "DBS1":
            row[0] = "DBS1"
        elif dName == "DBS2":
            row[0] = "DBS2"
        elif dName == "WPH1":
            row[0] = "WPH1"
        elif dName == "WPH2":
            row[0] = "WPH2"
        elif dName == "SH1":
            row[0] = "SH1"
        elif dName == "SH2":
            row[0] = "SH2"
        elif dName == "BPS":
            row[0] = "BPS"
        elif dName == "MPS":
            row[0] = "MPS"
        elif dName == "CPS":
            row[0] = "CPS"
        elif dName == "CER":
            row[0] = "CER"
        elif dName == "CNT":
            row[0] = "CNT"
        elif dName == "TRC":
            row[0] = "TRC"
        cursor.updateRow(row)


In [45]:
# Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','N2_Depot_Building_NAME']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))

arcpy.AssignDomainToField_management(output_feature, 'Type', domainList[0])
arcpy.AssignDomainToField_management(output_feature, 'SubType', domainList[1])
arcpy.AssignDomainToField_management(output_feature, 'Status', domainList[2])
arcpy.AssignDomainToField_management(output_feature, 'StructureType', domainList[3])
arcpy.AssignDomainToField_management(output_feature, 'StructureLevel', domainList[4])
arcpy.AssignDomainToField_management(output_feature, 'Name', domainList[5])

<Result 'N2_DEP_WS_merged'>

In [18]:
# Apply Symbology
symbolLyrx = r'C:/Users/emasu/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structures.lyrx'
arcpy.ApplySymbologyFromLayer_management(output_feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [19]:
# Alter field. Rename a field name 'ObjectId' to 'ObjectId1'. The GIS table has OBJECTID, which is reserved name.
# We cannot use this name.
arcpy.AlterField_management(output_feature, 'ObjectId', 'ObjectId1', 'ObjectId1', 'TEXT')


<Result 'Station_Structure_merged_Apalit'>

In [20]:
# Delete Fields
dropF = ['Bldg_Name', 'OmniClassDescription', 'temp']
arcpy.DeleteField_management(output_feature, dropF)

<Result 'Station_Structure_merged_Apalit'>

In [21]:
# Add 'Grid' and 'Zone' for monitoring. this is used in reference to P6, too
addFields = ['Grid', 'Zone', 'ID']

for field in addFields:
    arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE","")